In [1]:
import pandas as pd
ks = pd.read_csv('/home/moritz/Desktop/kaggle/kickstarter-projects/ks-projects-201801.csv',parse_dates=['deadline', 'launched'])
ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [2]:
print('Total number of cases to be considered: \n')
print(ks.values.shape[0])


Total number of cases to be considered: 

378661


In [3]:
print(pd.unique(ks.state))

['failed' 'canceled' 'successful' 'live' 'undefined' 'suspended']


In [4]:
ks.groupby('state')['ID'].count()

state
canceled       38779
failed        197719
live            2799
successful    133956
suspended       1846
undefined       3562
Name: ID, dtype: int64

In [5]:

ks=ks.query('state != "live"')
print(pd.unique(ks.state))

['failed' 'canceled' 'successful' 'undefined' 'suspended']


In [6]:
ks.groupby('state')['ID'].count()



state
canceled       38779
failed        197719
successful    133956
suspended       1846
undefined       3562
Name: ID, dtype: int64

In [7]:
print('Total number of cases to be considered finally : \n')
print(ks.values.shape[0])

Total number of cases to be considered finally : 

375862


In [8]:
# Add outcome column, "successful" == 1, others are 0
ks = ks.assign(outcome=(ks['state'] == 'successful').astype(int))


In [9]:

ks = ks.assign(hour=ks.launched.dt.hour,
               day=ks.launched.dt.day,
               month=ks.launched.dt.month,
               year=ks.launched.dt.year)

print(ks.head())




           ID                                               name  \
0  1000002330                    The Songs of Adelaide & Abullah   
1  1000003930      Greeting From Earth: ZGAC Arts Capsule For ET   
2  1000004038                                     Where is Hank?   
3  1000007540  ToshiCapital Rekordz Needs Help to Complete Album   
4  1000011046  Community Film Project: The Art of Neighborhoo...   

         category main_category currency   deadline     goal  \
0          Poetry    Publishing      GBP 2015-10-09   1000.0   
1  Narrative Film  Film & Video      USD 2017-11-01  30000.0   
2  Narrative Film  Film & Video      USD 2013-02-26  45000.0   
3           Music         Music      USD 2012-04-16   5000.0   
4    Film & Video  Film & Video      USD 2015-08-29  19500.0   

             launched  pledged     state  backers country  usd pledged  \
0 2015-08-11 12:12:28      0.0    failed        0      GB          0.0   
1 2017-09-02 04:43:57   2421.0    failed       15      US 

In [10]:


from sklearn.preprocessing import LabelEncoder

cat_features = ['category', 'currency', 'country']
encoder = LabelEncoder()

# Apply the label encoder to each column
encoded = ks[cat_features].apply(encoder.fit_transform)
encoded.head(10)



,category,currency,country
0,108,5,9
1,93,13,22
2,93,13,22
3,90,13,22
4,55,13,22
5,123,13,22
6,58,13,22
7,41,13,22
8,113,13,22
9,39,13,22


In [11]:
# These are the columns we are gonna use in our model training
# Since ks and encoded have the same index and I can easily join them
data = ks[['goal', 'hour', 'day', 'month', 'year', 'outcome']].join(encoded)
data.head()




,goal,hour,day,month,year,outcome,category,currency,country
0,1000.0,12,11,8,2015,0,108,5,9
1,30000.0,4,2,9,2017,0,93,13,22
2,45000.0,0,12,1,2013,0,93,13,22
3,5000.0,3,17,3,2012,0,90,13,22
4,19500.0,8,4,7,2015,0,55,13,22


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train ,y_test = train_test_split(data, ks['state'],test_size=0.2)

In [13]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [14]:
acc = model.score(X_test, y_test)

In [15]:
print(acc)

0.8899870964308993


In [16]:
predicted = model.predict(X_test)



In [19]:
predicted=predicted.reshape(-1,1)
predicted

array([['failed'],
       ['successful'],
       ['failed'],
       ...,
       ['failed'],
       ['successful'],
       ['failed']], dtype=object)

In [22]:
y_test=y_test.values.reshape(-1,1)

In [23]:
y_test



array([['failed'],
       ['successful'],
       ['failed'],
       ...,
       ['failed'],
       ['successful'],
       ['failed']], dtype=object)

In [24]:
for i in range(5):
    print(predicted[i], y_test[i])

['failed'] ['failed']
['successful'] ['successful']
['failed'] ['failed']
['failed'] ['failed']
['successful'] ['successful']
